##Mount Google Drive

In [4]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


##Importing Packages

In [5]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Reshape, Flatten, Conv2D, Dense, Conv2DTranspose, Dropout
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import Adam
import numpy as np
!mkdir generated_images resized_images

###Unzip the file in a destination directory

In [ ]:
!unzip /content/drive/MyDrive/DATASETS/paintings1k.zip -d paintings

In [7]:
images_path = "/content/paintings/"

##Resizing Data to feed NN

In [8]:
import os
import cv2

reshape_size = (128,128)

for i,image in enumerate(os.listdir(images_path)):
  img_path = os.path.join(images_path, image)
  img = cv2.imread(img_path)
  img = cv2.resize(img, reshape_size)
  cv2.imwrite(f"resized_images/{i}.png", img)

##NN Parameters

In [9]:
img_width = 128
img_height = 128
channels = 3
img_shape = (img_width, img_height, channels)
latent_dim = 100
adam = Adam(lr = 0.0002)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Since I need an output of 128, and I use 3 Conv2DTranspose I shoulde start the first layer with (((128/2)/2)/2) = 16

In [10]:
def build_generator():
  model = Sequential([
    Dense(256 * 16 * 16 , input_dim = latent_dim), #reshape the two 8 in case doent match
    LeakyReLU(alpha = 0.2),
    Reshape((16,16,256)), #reshape the two 8 in case doent match

    Conv2DTranspose(128, (4,4), strides = (2,2), padding = "same"),
    LeakyReLU(alpha = 0.2),
    
    Conv2DTranspose(128, (4,4), strides = (2,2), padding = "same"),
    LeakyReLU(alpha = 0.2),

    Conv2DTranspose(128, (4,4), strides = (2,2), padding = "same"),
    LeakyReLU(alpha = 0.2),

    Conv2D(3, (3,3), activation = "tanh", padding = "same") 
  ])

  model.summary()
  return model

generator = build_generator()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 65536)             6619136   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 65536)             0         
                                                                 
 reshape (Reshape)           (None, 16, 16, 256)       0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 32, 32, 128)      524416    
 nspose)                                                         
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 32, 32, 128)       0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 64, 64, 128)      262272    
 ranspose)                                              

In [11]:
def build_discriminator():
  model = Sequential([
    Conv2D(64, (3,3), padding = "same", input_shape = img_shape),
    LeakyReLU(alpha = 0.2),

    Conv2D(128, (3,3), padding = "same"),
    LeakyReLU(alpha = 0.2),

    Conv2D(128, (3,3), padding = "same"),
    LeakyReLU(alpha = 0.2),

    Conv2D(256, (3,3), padding = "same"),
    LeakyReLU(alpha = 0.2),

    Flatten(),
    Dropout(0.4),
    Dense(1, activation = "sigmoid")

  ])

  model.summary()
  return model

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 128, 128, 64)      1792      
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 128, 128, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 128)     73856     
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 128, 128, 128)     0         
                                                                 
 conv2d_3 (Conv2D)           (None, 128, 128, 128)     147584    
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 128, 128, 128)     0         
                                                                 
 conv2d_4 (Conv2D)           (None, 128, 128, 256)    

##Creating the GAN

In [12]:
GAN = Sequential([
  generator,
  discriminator
])

GAN.layers[1].trainable = False
GAN.summary()
GAN.compile(loss = "binary_crossentropy", optimizer = adam)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 128, 128, 3)       7671555   
                                                                 
 sequential_1 (Sequential)   (None, 1)                 4712705   
                                                                 
Total params: 12,384,260
Trainable params: 7,671,555
Non-trainable params: 4,712,705
_________________________________________________________________


##Output Images

In [13]:
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name = 0.00000000

def save_imgs(epoch):
    r, c = 2, 2
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    global save_name
    save_name += 0.00000001
    # print("%.8f" % save_name)

    # Rescale images 0 - 1
    # gen_imgs = 0.5 * gen_imgs + 0.5
    gen_imgs = (gen_imgs + 1) / 2.0
    # gen_imgs = gen_imgs * 255

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("generated_images/%.8f.png" % save_name)
    plt.close()

##Training GAN

In [14]:
from PIL import Image

array = []
path = "/content/resized_images/"

for image in os.listdir(path):
  image = Image.open(path + image)
  data = np.asarray(image)
  array.append(data)

X_train = np.array(array)
X_train = X_train /127.5 -1.
print(X_train.shape)

(1274, 128, 128, 3)


Remeber the batch size should be proportional to the amoubt of data

In [ ]:
def train(epochs, batch_size = 32):
  bat_per_epo = int(X_train.shape[0] / batch_size)

  valid = np.ones((batch_size, 1))
  fakes = np.zeros((batch_size, 1))

  for epoch in range(epochs):
    for j in range(bat_per_epo):
      #Get random batch
      idx = np.random.randint(0,X_train.shape[0], batch_size)
      imgs = X_train[idx]

      #Generate Fake Images
      noise = np.random.normal(0,1, (batch_size, latent_dim))
      gen_imgs = generator.predict(noise)

      #train discriminator
      d_loss_real = discriminator.train_on_batch(imgs, valid)
      d_loss_fake = discriminator.train_on_batch(imgs,fakes)
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

      noise = np.random.normal(0,1, (batch_size, latent_dim))

      #inverse y label
      g_loss = GAN.train_on_batch(noise,valid)

      print("******* %d %d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch,j, d_loss[0], 100* d_loss[1], g_loss))

    save_imgs(epoch)

train(60000, batch_size = 32)


##Create a GIF

In [ ]:
# Display a single image using the epoch number
# def display_image(epoch_no):
#   return PIL.Image.open('generated_images/%.8f.png'.format(epoch_no))

anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('generated_images/*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

##Saving models weights

In [ ]:
generator.save_weights("/content/drive/MyDrive/models/generator1hour.h5")
discriminator.save_weights("/content/drive/MyDrive/models/discriminator1hour.h5")